In [11]:
import numpy as np

class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [12]:
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=20)

In [14]:
mf.train()

Iteration: 10 ; error = 0.5543
Iteration: 20 ; error = 0.0624


[(0, 5.464480795304525),
 (1, 4.648584909766637),
 (2, 3.537319214419717),
 (3, 2.505280412992119),
 (4, 1.9041307803152083),
 (5, 1.4496764770159332),
 (6, 1.1347794379493068),
 (7, 0.8877091690507917),
 (8, 0.7034256873479929),
 (9, 0.554292475861226),
 (10, 0.4320262385562388),
 (11, 0.33205621415674647),
 (12, 0.25816116500444314),
 (13, 0.19987327024858503),
 (14, 0.15787686795531874),
 (15, 0.12585921978008033),
 (16, 0.10257979001079313),
 (17, 0.08536476861850759),
 (18, 0.0716750234340706),
 (19, 0.062416565656910734)]

In [15]:
mf.full_matrix()

array([[4.99010054, 2.99261899, 4.81674308, 1.00966077],
       [3.99503087, 0.93193798, 4.38743446, 1.0120055 ],
       [1.00949775, 0.9985783 , 3.26383177, 4.98272164],
       [1.01127685, 0.91598731, 2.93002938, 3.99160949],
       [3.44049435, 1.02799821, 4.95603105, 3.98674524]])

In [18]:
print(mf.P.shape)
mf.P

(5, 2)


array([[ 0.8942867 , -1.43932163],
       [-0.23644101, -1.17383341],
       [-0.09261874,  1.129838  ],
       [ 0.15590024,  0.81617387],
       [-0.85120859, -0.04377775]])

In [19]:
print(mf.Q.shape)
mf.Q

(4, 2)


array([[ 0.18134653, -1.37203293],
       [ 1.41673221, -0.12424322],
       [-0.12474672, -0.54467819],
       [ 0.01074064,  1.65825436]])